### The purpose of this notebook is to help build a Training Loop class

We will use the existing **modules.lib.ChextXRayImages** class to obtain the DataFrames, Datasets and Loader for the CheXpert dataset.

We will create a dummy NN model just to validate that our training loop is good.  

We will use the *n_random_rows* parameter in our Loaders class to make the training loops quick.

#### We want to flesh out a few things in this notebook to help us build the class:
- Make sure out model output and our loss function are compatible (i.e. who does the sigmoid/softmax)
- Find an adequate way to display a helpful "accuracy" score on the completion of each epoch
- Build objects that hold the history of the training (loss, weights, predictions)
- Add the ability to tag the predictions back to the original DataFrame via the ImageID


In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

In [2]:
import numpy as np
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from modules.lib.ChextXRayImages import *
from modules.models.CustomPneumonia import CustomPneumoniaNN

from PIL import Image
import copy

import torch.optim as optim
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage
import torchvision.models as models

from torchsummary import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

#### Let's run in cuda so we can catch things like:
TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

Working on device=cuda


### modules.lib.ChextXRayImages

We will use this class to get both training and validation data loaders.

We also want to pull the list of target columns and the 2 DataFames used in the loaders.

The latter will be used to join our prediction values so that we know which predictions go with which x-rays.

*Note:  The Loaders class will display a warning if there is more than a 2% difference in feature occurrence between train and validation.  Since we are using a very small sample of the full set of rows, we would expect warning on one or a few features.  As the number of rows in the sample goes up, the odds of getting an imbalance warning become fairly low.*

*See the bottom of EDA.ipynb for more details*


In [4]:
loaders = Loaders()
batch_size=16
val_percent=0.15
number_images = 1000
train_loader, val_loader = loaders.getDataTrainValidateLoaders(batch_size=batch_size, 
                                                                        val_percent=val_percent, 
                                                                        n_random_rows=number_images)

target_columns = loaders.target_columns

train_actual = loaders.train_df
val_actual = loaders.val_df

print(f'Number of Training Batches: {len(train_loader):,}')
print(f'Number of Validation Batches: {len(val_loader):,}')
print(f'Number of Training Images: {len(train_loader) * batch_size:,}')
print(f'Number of Validation Images: {len(val_loader) * batch_size:,}')

C:\git\Springboard-Public\Capstone Project 2\modules\lib\ChextXRayImages.py:244: UserWarning: 
Feature Imbalance Detected (train % - val %):
   Lung_Opacity: 3.24%
   Pneumothorax: 2.30%

  self.warnFeatureImbalance(train, value)


Number of Training Batches: 53
Number of Validation Batches: 10
Number of Training Images: 848
Number of Validation Images: 160


### Simple FC only model used just for testing

We do not expect any kind of usable results from this model.  

The biggest aspect here is the output shape and any final activation functions.

In [5]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)       
        self.flattened_length_ = 1*320*320
        self.fc1 = nn.Linear(self.flattened_length_, 12)
       
    def forward(self, x):    
        x = x.view(-1, self.flattened_length_)    
        x = self.fc1(x)
        return x

In [6]:
net = SimpleModel()

net = nn.DataParallel(net)
net.to(device)

summary(net, (1, 320, 320)) #Known Harded code size generated by data loaders (todo: make attribute of loaders)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 12]       1,228,812
       SimpleModel-2                   [-1, 12]               0
Total params: 1,228,812
Trainable params: 1,228,812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.39
Forward/backward pass size (MB): 0.00
Params size (MB): 4.69
Estimated Total Size (MB): 5.08
----------------------------------------------------------------


### Look at output from Loaders

We just need to look at the first x-ray from the loader.

#### We are looking here for:
- What structure the loader gives us
- The shapes of these objects
- The shape and content of the output from our test model

In [7]:
data = next(iter(train_loader))
ImageID, inputs, labels = data['id'], data['img'], data['labels']

print('Batch ImageIDs: ', ImageID.detach().numpy())

print('\n' + '-' * 50)

# move data to device GPU OR CPU
inputs = inputs.to(device)
labels = labels.to(device)

outputs = net(inputs)

print('labels shape (batch size, feature count): ', labels.shape)
print('inputs shape (batch size, channels, w, h): ', inputs.shape)
print('outputs shape (batch size, feature count): ', outputs.shape)

print('\n' + '-' * 50)

print('labels:\n', labels)

print('\n' + '-' * 50)

print('model output:\n', outputs)

Batch ImageIDs:  [204526 111147  42219 119536 189227 188489 164619 179090 143958 187657
  12808 124342 146833 200985 117203 199989]

--------------------------------------------------
labels shape (batch size, feature count):  torch.Size([16, 12])
inputs shape (batch size, channels, w, h):  torch.Size([16, 1, 320, 320])
outputs shape (batch size, feature count):  torch.Size([16, 12])

--------------------------------------------------
labels:
 tensor([[0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0

#### As we can see, the labels and the model output have the same shape

But the **labels are Boolean** and the model **output are Real** numbers.

Our plan is to use **BCEWithLogitsLoss** as our loss function.  

This function takes the **sigmoid** result for each of the outputed feature predictions and the performs **binary cross-entropy** for each of these squashed values.

Because of this, we want the output to be Real values.

*Note:  If down the road we want to try different loss functions, we may have to find a dynamic way to connect our model output to our loss function.  But for now, we will leave this "hard coded".*


## Accuracy

Since our goal is to store all the interim losses and predictions for each epoch, we should be able to look at accuracy, recall and precision very methodically after the training completes.

But we still need a way to monitor how well our model is performing in real-time.  We will be trying several different models.  We will probably want to change parameters on the model with the ability to abort the training if things aren't looking good.  So having a reliable indicator of accuracy during training is critical.

### Overall Accuracy

We have 12 features.  If we take $n$ x-rays, we will have $12n$ predictions.

So a simple accuracy approach could be take all the percent of correct predictions:

### $accuracy = \frac{TotalCorrect}{12n}$

But of we look at the EDA notebook, we will see that the actual positive rate for all but 3 of the features is 15% or less.

This means that non-positive finding will dominate the accuracy score.

i.e. A bad model will look pretty good at predicting non-positive results.

In [8]:
data = next(iter(train_loader))
ImageID, inputs, labels = data['id'], data['img'], data['labels']

# move data to device GPU OR CPU
inputs = inputs.to(device)
labels = labels.to(device)

outputs = net(inputs)

# Since the model ouputs raw Real numbers, we need to convert the output to Boolean values

predicted = torch.sigmoid(outputs.data) 
predicted[predicted >= 0.5] = 1 # assign 1 label to those with less than 0.5
predicted[predicted < 0.5] = 0 # assign 0 label to those with less than 0.5

print('Actual:\n', labels)
print('\n' + '-' * 50)
print('Predicted:\n', predicted)

print('\n' + '-' * 50)

train_batch_size, train_label_count = labels.shape

print('Accurate Predictions: ', (predicted == labels).sum().item())
print('Total Predictions: ', train_batch_size * train_label_count)
train_acc = float((predicted == labels).sum()) / float((train_batch_size * train_label_count))
print(f'Overall Accuracy: {train_acc:.2%}')

Actual:
 tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

--------------------------------------------

### We need a better score

Our test model is probably pretty close to random choise, but it still shows an 80% accuracy because of the sparseness of positive findings in most of the features.

Since we will want to look at scores like sensitivity, precision, F1 etc. when the training is done, so why not use some of these concepts in real-time output during training?

#### Let's take a quick look at what we can do with SKLearn:

We want to look at the **average** parmaeter for SKL's recall, precision and F1 to make sure we are using the right one for multi-label classification

#### We first need to get an understanding how these scores work with multi-lable classification

The primary parameter will look at for all 3 of these scores is **average**.  There are 5 options:
- average=None
- average='micro'
- average='macro'
- average='samples'
- average='weighted'

None will return a list of scores, one for each target.  All there other will return a scalar based on how the average is set.

Since all 3 scores work the same, we will just look at recall.

*Note:  Since F1 is just the harmonic meand of recall and precision, we will not show this value in our epoch output.*

In [9]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, hamming_loss

y_true = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
                   [0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
                   [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.]])

y_pred = np.array([[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
                   [1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
                   [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
                   [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
                   [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
                   [1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0],
                   [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1],
                   [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
                   [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                   [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
                   [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
                   [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]]) 

true_positive_count = y_true.sum(axis=0)

# average=None will show a seperate score for each target
itemized_recall = recall_score(y_true=y_true, y_pred=y_pred, average=None)

# Let's put this 
df_itemized = pd.DataFrame({'Target':target_columns, 
                            'True Positive Count':true_positive_count, 
                            'Recall':itemized_recall})

display(df_itemized)

### Let's build these scores manually to see how they work ###

#Average (macro)
average_itemized_recall = np.mean(itemized_recall)

#Weighted Average (micro and weighted)
weighted_average_itemized_recall = sum(itemized_recall * true_positive_count) / true_positive_count.sum()
   
"""
Per Row (sample):  
This caclation differes from the rest.  
The others caclulate the recall for all rows by target. They then average the target recall values.
This one caculates the recall for all the targets by row (or samle).  It then averages the row recall values.              
"""                   
row_recalls = []
for i in range(y_true.shape[0]):
    tp=0
    fn=0
    for j in range(y_true.shape[1]):
        t = y_true[i,j]
        p = y_pred[i,j]
        if t == 1 and p == 1:
            tp+=1
        if t == 1 and p == 0:
            fn+=1
    if (tp + fn) > 0:
        row_recall = tp / (tp + fn)
        row_recalls.append(row_recall)

row_recalls = np.array(row_recalls)
per_row_recall = row_recalls.sum() / y_true.shape[0]

print('Manual:')
print('\n' + '-' * 50)
print('\nItemized avg',average_itemized_recall)
print('\nItemized weighted avg',weighted_average_itemized_recall)
print('\nItemized per row avg',per_row_recall)

print('\n\nSKLearn:')
print('\n' + '-' * 50)
print('\naverage=macro: ',recall_score(y_true=y_true, y_pred=y_pred, average='macro'))
print('\naverage=micro: ',recall_score(y_true=y_true, y_pred=y_pred, average='micro'))
print('\naverage=weighted: ',recall_score(y_true=y_true, y_pred=y_pred, average='weighted'))
print('\naverage=samples: ',recall_score(y_true=y_true, y_pred=y_pred, average='samples'))

C:\Users\youci\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Target,True Positive Count,Recall
0,Enlarged_Cardiomediastinum,0.0,0.000000
1,Cardiomegaly,5.0,0.400000
2,Lung_Opacity,6.0,0.166667
3,Lung_Lesion,2.0,0.500000
4,Edema,7.0,0.571429
5,Consolidation,0.0,0.000000
6,Pneumonia,0.0,0.000000
7,Atelectasis,1.0,1.000000
8,Pneumothorax,1.0,0.000000
9,Pleural_Effusion,7.0,0.142857


Manual:

--------------------------------------------------

Itemized avg 0.23174603174603173

Itemized weighted avg 0.3333333333333333

Itemized per row avg 0.19583333333333333


SKLearn:

--------------------------------------------------

average=macro:  0.23174603174603173

average=micro:  0.3333333333333333

average=weighted:  0.3333333333333333


C:\Users\youci\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



average=samples:  0.1958333333333333


### We also want to look at accuracy and Hamming loss

SKLearn's accuracy_score is very unforgiving.  It only counts rows that have every category correct.

So chances are, this score will not prove very useful, but it can still show us trends.

The Hamming loss looks at each target independantly, so event partial label matches can contribute.

*Note:  Hamming loss is negated, so we want the score to be as small as possible.*

In [10]:
print('accuracy_score: ',accuracy_score(y_true=y_true, y_pred=y_pred, normalize=True))
print('\nhamming_loss: ',hamming_loss(y_true=y_true, y_pred=y_pred))

accuracy_score:  0.0

hamming_loss:  0.5


In [11]:
y_true = labels.cpu().data.numpy()

y_pred = np.random.choice(a=[0, 1], size=predicted.shape)
y_true, y_pred

# itemized_recall = recall_score(y_true=y_true, y_pred=y_pred, average=None)
# itemized_precision = precision_score(y_true=y_true, y_pred=y_pred, average=None)
# itemized_f1 = f1_score(y_true=y_true, y_pred=y_pred, average=None)
# tru_positive_count = y_true.sum(axis=0)
# df_itemized = pd.DataFrame({'Target':target_columns, 
#                             'True Positive Count':tru_positive_count, 
#                             'Recall':itemized_recall, 
#                             'Precision':itemized_precision, 
#                             'F1':itemized_f1})
# display(df_itemized)

# sensitivity = recall_score(y_true=y_true, y_pred=y_pred, average='samples')
# precision = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
# f1 = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

# label_sensitivity = recall_score(y_true=y_true, y_pred=y_pred, average=None)
# label_precision = precision_score(y_true=y_true, y_pred=y_pred, average=None)
# label_f1 = f1_score(y_true=y_true, y_pred=y_pred, average=None)

# print('Random:\n' + '-' * 50)
# print(label_sensitivity)
# print(f'Sensitivity: {sensitivity:.2%}')
# print(f'Precision: {precision:.2%}')
# print(f'F1 (harmonic avarage): {f1:.2%}')

# y_pred = predicted.cpu().data.numpy()

# sensitivity = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')
# precision = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
# f1 = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

# print('\n\nTest Model:\n' + '-' * 50)
# print(f'Sensitivity: {sensitivity:.2%}')
# print(f'Precision: {precision:.2%}')
# print(f'F1 (harmonic avarage): {f1:.2%}')

(array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1],
        [

In [12]:
def parseLoaderData(data):
    """
    The data loaders output a dictionary with 3 keys
    The first 2 keys hold single values for the ImageID and the actual tensor of the image
    The last key holds the ground truth vector of the 12 lables
    """ 
    
    ids, inputs, labels = data['id'], data['img'], data['labels']
    # move data to device GPU OR CPU
    inputs, labels = inputs.to(device), labels.to(device)
    return ids, inputs, labels

In [13]:
def getPredictionsFromOutput(outputs):
    """
    We are using BCEWithLogitsLoss for out loss
    In this loss funciton, each label gets the sigmoid (inverse of Logit) before the CE loss
    So our model outputs the raw values on the last FC layer
    This means we have to apply sigmoid to our outputs to squash them between 0 and 1
    We then take values >= .5 as Positive and < .5 as Negative 
    """
    
    predictions = torch.sigmoid(outputs.data) 
    predictions[predictions >= 0.5] = 1 # assign 1 label to those with less than 0.5
    predictions[predictions < 0.5] = 0 # assign 0 label to those with less than 0.5   
    return predictions

In [14]:
def updatePredictions(dictionary, ids, predictions):
    """
    Keep track of predictions using the same index as our DataFrame
    This will allow us to compare to the actual labels
    
    We only are taking the last prediction for each x-ray, but we could extend this later if wanted.
    """
    
    for i in range(len(ids)):
        id = ids[i].item()    
        dictionary[id] = [int(f.item()) for f in predictions[i]]

In [15]:
def processBatch(net, data, optimizer=None):
    """
    Used for both training and validation.
    Validation will not pass in the optimizer.
    """

    # Convert output from loader
    ids, inputs, labels = parseLoaderData(data)
    
    if optimizer:
        # zero the parameter gradients
        optimizer.zero_grad()
        
    # Convert output to predicitons
    outputs = net(inputs)
    predictions = getPredictionsFromOutput(outputs)
    
    return ids, inputs, labels, outputs, predictions 

In [16]:
def backProp(criterion, outputs, labels, optimizer):
    """
    Get loss value from criterion
    run backprop on the loss
    update weights in optimizer
    update epoch loss
    """
    
    loss = criterion(outputs, labels)#.float())
    loss.backward()
    optimizer.step()
    return loss.item()

In [17]:
def getPredictionDataFrame(epoch_predictions):
    result = pd.DataFrame(epoch_predictions).transpose()
    result.columns = target_columns
    return result

In [18]:
epoch_loss = 0
losses_hx = {}

train_prediction_hx = {}
val_prediction_hx = {}

epoch_train_predictions = {}
epoch_val_predictions = {}

df_train_prediction = None
df_val_prediction = None

In [19]:
def closeTrainEpoch(i):
    global training_time_elapsed
    global epoch_loss
    global losses_hx
    global train_prediction_hx
    global last_train_predictions
    global df_train_prediction
    
    training_time_elapsed = datetime.now() - start_time
    epoch_loss = epoch_loss / len(train_loader)
    losses_hx[i] = epoch_loss    
    
    df_train_prediction = getPredictionDataFrame(epoch_train_predictions)
    train_prediction_hx[i] = df_train_prediction
    last_train_predictions = {}

In [20]:
def closeValEpoch(i):
    global validation_time_elapsed
    global val_prediction_hx
    global last_val_predictions
    global df_val_prediction
    
    validation_time_elapsed = datetime.now() - start_time
    
    df_val_prediction = getPredictionDataFrame(epoch_val_predictions)
    val_prediction_hx[i] = df_val_prediction
    last_val_predictions = {}

In [21]:
learning_rate = 1e-4
num_epochs = 2

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)#, weight_decay=0.9)

In [22]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    start_time = datetime.now()
    
    
    # Training
    net.train()
    for i, data in enumerate(train_loader, 0):
        ids, inputs, labels, outputs, predictions = processBatch(net, data, optimizer)
        updatePredictions(epoch_train_predictions, ids, predictions)
        epoch_loss += backProp(criterion, outputs, labels, optimizer)

    closeTrainEpoch(i)
    
    
    # Validation
    net.eval()
    with torch.no_grad():
      for data in val_loader:          
            ids, inputs, labels, _, predictions = processBatch(net, data)
            updatePredictions(epoch_val_predictions, ids, predictions)
   
    closeValEpoch(i)
    
    
    # stdout Results
    print(f'Epoch [{epoch+1}/{num_epochs}], \
\n          Epoch Loss: {epoch_loss:.4f} \
\n          Training Time: {training_time_elapsed})  \
\n          Validation Time: {validation_time_elapsed})')

Epoch [1/2], 
          Epoch Loss: 0.9844 
          Training Time: 0:00:03.043863)  
          Validation Time: 0:00:03.589405)
Epoch [2/2], 
          Epoch Loss: 0.8000 
          Training Time: 0:00:03.036882)  
          Validation Time: 0:00:03.583421)


In [23]:
def displayImageResults(actual, predicted, imageID):
    actual = actual[target_columns].transpose()
    predicted = predicted.transpose()
    result = pd.DataFrame()
    result['Actual'] = actual[imageID]
    result[result['Actual']==-1] = 0
    result['Predicted'] = predicted[imageID]
    result['Successful'] = result['Actual'] == result['Predicted']
    display(result)

In [24]:
displayImageResults(train_actual, 
                    df_train_prediction, 
                    45510)

,Actual,Predicted,Successful
Enlarged_Cardiomediastinum,0,0,True
Cardiomegaly,0,0,True
Lung_Opacity,0,0,True
Lung_Lesion,0,0,True
Edema,0,0,True
Consolidation,0,0,True
Pneumonia,0,0,True
Atelectasis,0,1,False
Pneumothorax,0,0,True
Pleural_Effusion,1,1,True


In [25]:
df_train_result = train_actual.join(df_train_prediction, lsuffix='_actual', rsuffix='_predicted')
df_val_result = val_actual.join(df_val_prediction, lsuffix='_actual', rsuffix='_predicted')

In [26]:
df_train_result 
df_val_result

,PatientID,StudyID,Age,Sex_Male,Sex_Unknown,Orientation_PA,Support Devices,Image_Path,Hierarchical_Path,Enlarged_Cardiomediastinum_actual,...,Lung_Opacity_predicted,Lung_Lesion_predicted,Edema_predicted,Consolidation_predicted,Pneumonia_predicted,Atelectasis_predicted,Pneumothorax_predicted,Pleural_Effusion_predicted,Pleural_Other_predicted,Fracture_predicted
ImageID,,,,,,,,,,,,,,,,,,,,,
79521,19098,1,68,0,0,1,0.0,data/raw/train/patient19098/study1/view1_front...,data/d21/d48/i79521.jpg,0,...,1,0,0,0,0,1,0,1,1,1
8401,2079,1,71,0,0,1,0.0,data/raw/train/patient02079/study1/view2_front...,data/d1/d29/i8401.jpg,0,...,0,1,1,1,1,1,0,1,0,1
77894,18714,4,58,1,0,0,0.0,data/raw/train/patient18714/study4/view1_front...,data/d44/d14/i77894.jpg,0,...,0,1,0,1,1,1,0,0,0,0
12169,3031,1,59,1,0,1,0.0,data/raw/train/patient03031/study1/view1_front...,data/d19/d31/i12169.jpg,0,...,0,0,0,0,0,0,0,0,0,1
187761,44734,3,71,0,0,0,1.0,data/raw/train/patient44734/study3/view1_front...,data/d11/d34/i187761.jpg,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191877,46242,3,66,0,0,0,1.0,data/raw/train/patient46242/study3/view1_front...,data/d27/d42/i191877.jpg,0,...,0,0,1,0,0,0,0,0,0,0
223323,64462,1,49,0,0,0,1.0,data/raw/train/patient64462/study1/view1_front...,data/d23/d12/i223323.jpg,0,...,1,0,1,0,0,0,0,1,0,0
136247,32694,2,51,0,0,0,1.0,data/raw/train/patient32694/study2/view1_front...,data/d47/d44/i136247.jpg,0,...,0,0,0,0,0,0,0,0,0,1
